### Resnet

레이어가 많아져 신경망이 깊어질수록 기울기 소실/ 폭발 문제가 커진다  
깊은 레이어까지 학습이 잘되도록 하는 방법이 shortcut(Skip) Connection 입니다

low-level vision과 computer graphics 분야에서 Partial Differential Equations(PDEs)를 풀기 위해   
Multigrid 방법들이 많이 사용됩니다.  
이것은 다양한 스케일(크기)에서의 하위문제로 시스템을 재정의하는 것입니다.    
쉽게 설명하면, 코카콜라 캔을 학습한다고 하였을 때, 코카콜라 캔이 가까이에서 찍힌 것은 크게 나타나고, 멀리서 찍힌 것은 작게 나타납니다.   
이러한 스케일의 변화에 대응하기 위한 컴퓨터 비전 기술은 pyramid와 같은 형태의 multigrid를 만들어서 해결합니다.


그런데 다른점은 무엇이냐면, 이전에는 multigrid를 만들어서 (이미지 크기를 다양하게 하거나, 필터 크기를 다양하게 해서) 각각을 계산하는 방식을 사용하였습니다.   
기존의 CNN 구조에서는 이전의 것이 다음으로 전달되어 영향을 미치게됩니다.   
입력부분에 가까운 하위 레이어에서는 매우 단순한 구조나 노이지한 패턴이 보이는 low-level feature가 학습이되고, 출력부분에 가까운 상위 레이어에서는 구조적인 부분이 학습되는 high-level feature가 학습됩니다.

그런데 앞선 부분의 feature가 뒤쪽까지 영향이 직접적으로 전달되는 것이 아니라, 중간을 거쳐 전달되기 때문에 학습의 과정에서 크게크게 변합니다.  
그런데 shortcut connection을 추가해주게 되면 (수식적으로) 이전으로부터 얼만큼 변하는지 나머지(residual)만 계산하는 문제로 바뀌게 됩니다.  
즉, 현재 레이어의 출력값과 이전 스케일의 레이어 출력값을 더해 입력을 받기 때문에,   
그 차이를 볼 수 있게 되는 것이죠.  
따라서 학습하는 과정에서 그 '조금'을 하면 되는 것이고, 더 빠르게 학습한다는 장점이 생깁니다!

이전에 얕은 모델과 깊은 모델을 비교했을 때, 깊은 모델이 더 안좋아진다고 했었습니다.   
그런데 Kaiming He는 그래서는 안된다고 했습니다.  
그런 문제점을 identity mapping이라는 것을 통해 꼬집었습니다.   
얕은 모델에서 단순히 아무것도 하지 않는 layer인(convolution을 통과하지 않고 값을 전달하는)  
identity mapping을 쌓으면(덧셈 연산으로)   
얕은 모델 그대로의 성능을 나타낼 것이라는 자명한 사실에 하나의 가정을 더합니다.  
"쌓여있는 레이어가 underlying mapping을 fit하는 것보다 residual mppaing을 fit하는 것이 쉽다."   
그리고 shortcut connection이 이 역할을 정확하게 할 수 있다고 말합니다.   
즉, 이전에 학습된 모델(레이어들)의 출력과 추가된 레이어의 출력의 차이값인 나머지(residual)만 학습하면 되기에 연산이 간단해지고, error값 크기의 측면에서 학습이 더 쉽다는 것입니다.

identity mapping은 입력값을 그대로 전달한다는 의미에서 identity입니다.  
위의 shortcut connection에서 identity로 표현한 것처럼,   
shortcut connection과 identity mapping은 다른 것이 아니라,   
의미적으로 identity는 값을 그대로 보낸다는 것입니다. :)

1. 이미지에서는 H(x) = x가 되도록 학습시킨다.

2. 네트워크의 output F(x)는 0이 되도록 학습시킨다.

3. F(x)+x=H(x)=x가 되도록 학습시키면 미분해도 F(x)+x의 미분값은 F'(x) + 1로 최소 1이상이다.

4. 모든 layer에서의 gradient가 1+F'(x)이므로 gradient vanishing현상을 해결했다.

In [ ]:
''' 처음을 제외하고는 균일하게 3 x 3 사이즈의 컨볼루션 필터를 사용했다
그리고 특성맵의 사이즈가 반으로 줄어들 때 특성맵의 뎁스를 2배로 높임'''

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import os, random, time
import copy

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
# *------- Basic setup -------*
import numpy as np
import pandas as pd
import os, random, time
import copy
from tqdm.notebook import tqdm
from multiprocessing import cpu_count
import matplotlib.pyplot as plt
from PIL import Image

# *------- torch -------*
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
#import torchvision.transforms as transforms
from torchsummary import summary

# *------- albumentations -------*
#!pip install albumentations==1.0.3
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# *------- sklearn -------*
from sklearn.model_selection import train_test_split

# *------- path -------*
base_path = "./"

# test 폴더를 한 단계 상위 폴더로 옮겨서 사용했습니다.
# 옮기지 않은 경우
#test_path = os.path.join(base_path, "test","test")
test_path = os.path.join(base_path, "test")
train_path = os.path.join(base_path, "train")

In [ ]:
path = './train/train_data.csv'

df = pd.read_csv(path)
df

In [ ]:
train_file_name = df['filen_name']
train_label = df['label']

# image 파일을 불러온뒤 변수에 저장


In [ ]:
tr_csv = pd.read_csv(os.path.join('./', "train","train_data.csv"))
tr_csv['path'] = tr_csv['filen_name'].apply(
    lambda x: os.path.join('train',x))

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(
    tr_csv['path'].values, tr_csv["label"].values, test_size=0.2, shuffle=True)

In [ ]:
def compute_acc(true, pred):
    return sum(true == pred) / len(true)

In [ ]:
def get_train_transforms():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.OneOf([A.Rotate(limit=10),
                 A.RandomBrightness(),
                 A.CoarseDropout(),
                 A.Cutout(num_holes=8, max_h_size=1, max_w_size=1, fill_value=1),
                 ], p=1.0),
        ToTensorV2(p=1.0)
    ])
def get_valid_transforms():
    return ToTensorV2(p=1.0)

def get_inferecne_transforms():
    return ToTensorV2(p=1.0)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms
import torch.nn.functional as F

class MNIST(Dataset):
    def __init__(self,X=None, y=None,transforms = None):
            super().__init__()
            self.file_path_list = X
            self.labels = y 
            self.transforms = transforms
    def __getitem__(self,idx):
        image = Image.open(self.file_path_list[idx]).convert("RGB")
        image = np.array(image, dtype = np.float32)
        image /= 255
        if self.transforms:
            image = self.transforms(image=image)['image']
            
        if self.labels is not None:
            label = self.labels[idx]
            label = torch.tensor(label,dtype = torch.int64)
            return image, label
         
        return image
    
    def __len__(self):
        return len(self.file_path_list)

In [ ]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(1010)

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
IMG_SIZE = (28,28)
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_CLASSES = 10
NUM_EPOCHS = 50
NUM_CPU = cpu_count()

In [ ]:
class MNIST(Dataset):
    def __init__(self, X=None, y=None, transforms=None):
        super().__init__()
        self.X = X
        self.y = y
        self.transforms = transforms

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img_path = self.X[idx]
        img = Image.open(img_path).convert("RGB")
        img = np.array(img, dtype=np.float32)
        img /= 255
        
        if self.transforms:
            img = self.transforms(image=img)['image']
        
        if self.y is not None:
            label = self.y[idx]
            label = torch.tensor(label, dtype=torch.int64)
            return img, label
        else:
            return img

In [ ]:
train_data = MNIST(train_x,train_y,get_train_transforms())
test_data = MNIST(valid_x,valid_y,get_valid_transforms())
train_dl = DataLoader(train_data,batch_size =32, shuffle = True, num_workers = NUM_CPU)
test_dl = DataLoader(test_data,batch_size =32, shuffle = False, num_workers = NUM_CPU)

In [ ]:
len(train_data)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self,in_channels, out_channels, stride = 1):
        super().__init__()
        #Batch Norm에 bias가 포함되어 있음으로 conv2d는 bias=Flase로 설정
        
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride=stride,padding =1,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels,out_channels * BasicBlock.expansion, kernel_size =3 ,stride = stride, padding= 1 ,bias = False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion),
        )
        self.shortcut = nn.Sequential()
        self.relu = nn.ReLU()
        
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels :
            self.shortcut = nn.Sequential(
            nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size = 1, stride= stride, bias = False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )
    def forward(self,x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x
class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels*BottleNeck.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels*BottleNeck.expansion)
            )
            
    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x

In [ ]:
class Net(nn.Module):
    def __init__(self,block,num_block,num_classes = 10, init_weights = True):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Sequential(
        nn.Conv2d(3,64,kernel_size = 7, stride = 2,padding =3 ,bias =False),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 3, stride =2, padding =1)
        )
        self.conv2_x = self._make_layer(block,64,num_block[0],1)
        self.conv3_x = self._make_layer(block,128,num_block[1],1)
        
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(128 * block.expansion, num_classes)
        if init_weights:
            self._initialize_weights()
            
    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks -1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels,out_channels,stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)
    def forward(self,x):
        output = self.conv1(x)
        x = self.conv2_x(output)
        x = self.conv3_x(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,mode = 'fan_out',nonlinearity ='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias,0)
                elif isinstance(m,nn.BatchNorm2d):
                    nn.init.constant_(m.weight,1)
                    nn.init.constant_(m.bias,0)
                elif isinstance(m,nn.Linear):
                    nn.init.normal_(m.weight,0,0.01)
                    nn.init.constatnt_(m.bais,0)
def resnet18():
        return Net(BasicBlock,[2,2])
def resnet34():
        return Net(BasicBlock,[3,4,6,3])
net = resnet34()

In [ ]:
model = torchvision.models.resnet34(pretrained = False)

In [ ]:
from torchsummary import summary
net.to(device)
summary(net, (3,224,224), device = device.type)

In [ ]:
# 사전학습된 가중치를 가져오지 않도록 pretrained는 Fasle
model = torchvision.models.resnet34(pretrained = False)

# number of features in the input of the linear layer
num_ftrs = model.fc.in_features

# sets the number of features of the linear layer
model.fc = torch.nn.Linear(num_ftrs, NUM_CLASSES)

# parameters
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
model = model.to(device)

# model summary
summary(model, (3, IMG_SIZE[0], IMG_SIZE[1]), BATCH_SIZE)

In [ ]:
def train_model(model, criterion, optimizer, num_epochs, train_loader,val_loader):
    since = time.time()
    best_model = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        
        # train
        model.train()
        running_loss = 0.0
        running_corrects = 0

        
        for step, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
        print('Train Loss: {:.4f} Train Acc: {:.4f}'.format(epoch_loss, epoch_acc))

        # validate
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in val_loader:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            optimizer.zero_grad()
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / val_size
        epoch_acc = running_corrects.double() / val_size
        print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        print('-' * 30)
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model.state_dict())
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Val Acc: {:.4f}'.format(best_acc))
    model.load_state_dict(best_model)
    return model

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-5)
model = net.to(device)
param = list(net.parameters())


In [ ]:
model = train_model(net, criterion, optimizer, 50, train_dl, test_dl)

In [ ]:
from tqdm import tqdm

for Epoch in tqdm(range(30)):
    for batch, labels in train_dl:
        batch = batch.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        output = model(batch)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        acc = compute_acc(labels.detach().cpu().numpy(), output.detach().cpu().numpy().argmax(-1))
        
    if Epoch % 10 == 0 or Epoch == 29:
        print(f'Epoch {Epoch}, loss : {loss}, acc : {acc}')

In [ ]:
test_df = pd.read_csv('./test/test_data.csv') 
test_file_dir = './test/'

In [ ]:
test_mnist_dataset = MNIST(test_file_dir + test_df['file_name'])
test_mnist_loader = DataLoader(test_mnist_dataset, batch_size = 32)
preds = None

for test_batch in tqdm(test_mnist_loader):
    test_batch = test_batch.to(device)
    output = net(test_batch)
    
    digit_pred = output.detach().cpu().numpy().argmax(-1)
    if preds is None:
        preds = digit_pred
    else:
        preds = np.concatenate([preds,digit_pred])
        

In [ ]:
submission = pd.read_csv('./sample_submission.csv') # sample submission 불러오기

submission['label'] = preds

submission.to_csv('submission.csv', index=False)

In [ ]:
preds[4866]